## **Extracting PII Information from Documents**

I have an example PDF document that contains Personal Identifiable Information (PII).

Imagine a scenario where we need to either a) tag this information and/or b) scrub this information from the document. This is how we can do it using Pydantic, insturctor, and Open AI.

In [7]:
#!pip install openai --upgrade
#!pip install instructor

In [1]:
from src.utils import read_pdf, model_query
from src.models import PIIData

In [2]:
PII_TEMPLATE = """
You are an expert Personally Identifiable Information (PII) scrubbing model.
Personally identifiable information (PII) is any data that could potentially identify a specific individual.
Extract the PII data from the following document.
"""

EXAMPLE_DOCUMENT = read_pdf("exampleDoc.pdf")

GPT_MODEL = "gpt-3.5-turbo-0613"

pii_data = model_query(
    pydantic_class=PIIData,
    system_prompt=PII_TEMPLATE,
    query=EXAMPLE_DOCUMENT,
    model=GPT_MODEL,
)

## **Showing the PII Data**

In [3]:
print(pii_data.model_dump_json(indent=2))

{
  "pii_data": [
    {
      "index": 4,
      "data_type": "amount",
      "pii_value": "£999.99"
    },
    {
      "index": 5,
      "data_type": "bank_account_number",
      "pii_value": "12349876"
    },
    {
      "index": 8,
      "data_type": "national_insurance_number",
      "pii_value": "AB124321E"
    },
    {
      "index": 17,
      "data_type": "email",
      "pii_value": "jbloggs@gmail.com"
    }
  ]
}


## **Showing the document but with PII data obscured**

In [4]:
print(pii_data.scrub_data(EXAMPLE_DOCUMENT))

Example Documentation  
 
 
Dear Mr. Joe Bloggs,  
 
You have been found to be in arrears.  
The amount owed is <amount_0>. If you do not respond within 30 days we will take the amount owed 
from your Bank Account number <bank_account_number_1>.  
 
Our records show that your national insurance number is <national_insurance_number_2> – please confirm this in writing.  
 
Sincerely,  
Jane Doe  
 
Sent to: <email_3>  
Page Number: 1
